In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import re

In [23]:
url = 'https://itambacuri-mg.portaltp.com.br/consultas/pessoal/planocarreiras.aspx'
response = requests.get(url)

content = response.content


site = BeautifulSoup(content, 'html.parser')


navegador = webdriver.Chrome()

navegador.get('https://itambacuri-mg.portaltp.com.br/consultas/pessoal/planocarreiras.aspx')

In [3]:
for i in range(3):
    navegador.execute_script("window.scrollBy(0, 150)")

In [128]:
botoes = navegador.find_elements_by_css_selector('td > button > span')
botoes[1].click()

In [130]:
x = BeautifulSoup(navegador.page_source, 'html.parser')

In [ ]:
for i in range(10):
    botoes = navegador.find_elements_by_css_selector('td > button > span')
    botoes[i].click()
    sleep(3)

In [131]:
def format_text(x):
    padrao = r'(\w+)\s+(\d+-\d+-\d+)\s+(R\$\s[\d.,]+)'

    global texto_formatado

    texto_formatado = re.sub(padrao, r'\1; \2; \3\n', x.get_text()).replace("R$", "; R$").replace('\xa0', ' ').replace('\n', '').split(";")


def get_info(np):
    #cargos
    cargos = x.find('tr', {'id': f'ctl00_containerCorpo_grdData_DXDataRow{np}'})

    format_text(cargos)

    #pessoas
    z = x.find('td', {'id': f'ctl00_containerCorpo_grdData_tcdxdt{np}'})
    z = z.get_text().split("\n")

    global valores_seis_digitos 
    valores_seis_digitos = [valor for valor in z if valor[:6].isdigit()]

get_info(201)

In [132]:
def get_info(np, vs):
    #cargos
    cargos = vs.find('tr', {'id': f'ctl00_containerCorpo_grdData_DXDataRow{np}'})

    format_text(cargos)

    #pessoas
    z = vs.find('td', {'id': f'ctl00_containerCorpo_grdData_tcdxdt{np}'})
    z = z.get_text().split("\n")

    global valores_seis_digitos 
    valores_seis_digitos = [valor for valor in z if valor[:6].isdigit()]
    sleep(2)

In [555]:
# z = x.find('td', {'id': 'ctl00_containerCorpo_grdData_tcdxdt22'})
# z = z.get_text().split("\n")


# def format_text(x):
#     padrao = r'(\w+)\s+(\d+-\d+-\d+)\s+(R\$\s[\d.,]+)'

#     global texto_formatado

#     texto_formatado = re.sub(padrao, r'\1; \2; \3\n', x.get_text()).replace("R$", "; R$").replace('\xa0', ' ').replace('\n', '').split(";")


# valores_seis_digitos = [valor for valor in z if valor[:6].isdigit()]

#tabela['Cargo e Nivel'] = texto_formatado[0]
#tabela['Salario'] = texto_formatado[1]
#tabela

In [21]:
#tabela = pd.DataFrame(columns=['Matrícula', 'Nome', 'CPF', 'Contrato', 'Data', 'Status', 'Cargo e Nivel', 'Salario'])

In [114]:
if 'tabela' in locals():
    for item in valores_seis_digitos:
        codigo = re.findall(r'^\d{6}', item)[0]
        nome = re.findall(r'\d{6}([A-Z\s]+)\*{3}', item)[0].strip()
        cpf = re.search(r'\*\*\*\.(\d{3})\.\d{3}\-\*\*', item)[0]
        contrato = list(re.search(r'\*\*\*\.(\d{3})\.\d{3}\-\*\*(.+?)(\d{2}/\d{2}/\d{4})', item).span())
        contrato[0] += 14
        contrato[1] -= 10
        data = re.findall(r'(\d{2}/\d{2}/\d{4})', item)[0]
        if '\xa0' in item[0]:
            status = re.findall(r'(\xa0\w+)', item)[0].strip()

        padrao = r'(\w+)\s+(\d+-\d+-\d+)\s+(R\$\s[\d.,]+)'
        

        tabela = tabela.append({
            'Matrícula': codigo,
            'Nome': nome,
            'CPF': cpf,
            'Contrato': item[contrato[0]:contrato[1]],
            'Data': data,
            'Status': status
        }, ignore_index=True)
else:
    tabela = pd.DataFrame(columns=['Matrícula', 'Nome', 'CPF', 'Contrato', 'Data', 'Status', 'Cargo e Nivel', 'Salario'])
    for item in valores_seis_digitos:
        codigo = re.findall(r'^\d{6}', item)[0]
        nome = re.findall(r'\d{6}([A-Z\s]+)\*{3}', item)[0].strip()
        cpf = re.search(r'\*\*\*\.(\d{3})\.\d{3}\-\*\*', item)[0]
        contrato = list(re.search(r'\*\*\*\.(\d{3})\.\d{3}\-\*\*(.+?)(\d{2}/\d{2}/\d{4})', item).span())
        contrato[0] += 14
        contrato[1] -= 10
        data = re.findall(r'(\d{2}/\d{2}/\d{4})', item)[0]
        status = re.findall(r'(\xa0\w+)', item)[0].strip()

        padrao = r'(\w+)\s+(\d+-\d+-\d+)\s+(R\$\s[\d.,]+)'
        

        tabela = tabela.append({
            'Matrícula': codigo,
            'Nome': nome,
            'CPF': cpf,
            'Contrato': item[contrato[0]:contrato[1]],
            'Data': data,
            'Status': status
        }, ignore_index=True)

/tmp/ipykernel_9437/2773360197.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append({


In [115]:
tabela2 = tabela.tail(len(valores_seis_digitos))

tabela2.loc[:, 'Salario'] = texto_formatado[1]
tabela2.loc[:, 'Cargo e Nivel'] = texto_formatado[0]

tabela2

/tmp/ipykernel_9437/3492839169.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabela2.loc[:, 'Salario'] = texto_formatado[1]
/tmp/ipykernel_9437/3492839169.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabela2.loc[:, 'Cargo e Nivel'] = texto_formatado[0]


,Matrícula,Nome,CPF,Contrato,Data,Status,Cargo e Nivel,Salario
949,764703,MARILENE LOURENCO DA SILVA,***.516.536-**,Efetivo,02/06/2014,Ativo,PROF ED BASICA II PII-III-B,"R$ 24,14"


In [65]:
# tabela2.loc[:, 'Salario'] = texto_formatado[1]
# tabela2.loc[:, 'Cargo e Nivel'] = texto_formatado[0]

# tabela

In [62]:
def ttt():
    for i in range(1,9):
        while navegador.find_element_by_css_selector(f'a[data-args="PN{i}"]'):
            element = navegador.find_element_by_css_selector(f'a[data-args="PN{i}"]')
            element.click()
            sleep(3)

In [26]:
from selenium.common.exceptions import NoSuchElementException

for i in range(10):
    botoes = navegador.find_elements_by_css_selector('td > button > span')
    botoes[i].click()
    sleep(5)

    MAX_CLICKS = 10
    click_count = 0

    while True:
        try:
            elementos = navegador.find_elements_by_css_selector('a[data-args="PBN"]')
            # Exclui o último elemento da lista
            elementos = elementos[:-1]
            
            elementos[0].click()
            click_count += 1
            if click_count >= MAX_CLICKS:
                break
        except (NoSuchElementException, IndexError):
            break

        sleep(5)


    if i == 199:
        element = navegador.find_element_by_xpath('//*[@id="ctl00_containerCorpo_grdData_DXPagerBottom"]/ul/li[4]/a')
        element.click()
        navegador.execute_script("window.scrollTo(0, 150);")
        sleep(5)

In [27]:
    
def rola():
    MAX_CLICKS = 10
    click_count = 0

    while True:
        try:
            elementos = navegador.find_elements_by_css_selector('a[data-args="PBN"]')
            # Exclui o último elemento da lista
            elementos = elementos[:-1]
            
            elementos[0].click()


            
            click_count += 1
            if click_count >= MAX_CLICKS:
                break
        except (NoSuchElementException, IndexError):
            break

        sleep(5)

In [93]:
url = 'https://itambacuri-mg.portaltp.com.br/consultas/pessoal/planocarreiras.aspx'
response = requests.get(url)

content = response.content


site = BeautifulSoup(content, 'html.parser')


navegador = webdriver.Chrome()

navegador.get('https://itambacuri-mg.portaltp.com.br/consultas/pessoal/planocarreiras.aspx')

for i in range(3):
    navegador.execute_script("window.scrollBy(0, 150)")

In [135]:
# Define ação para ignorar os alertas futuros
#warnings.simplefilter(action='ignore', category=FutureWarning)


def format_text(x):
    padrao = r'(\w+)\s+(\d+-\d+-\d+)\s+(R\$\s[\d.,]+)'

    global texto_formatado

    texto_formatado = re.sub(padrao, r'\1; \2; \3\n', x.get_text()).replace("R$", "; R$").replace('\xa0', ' ').replace('\n', '').split(";")

    sleep(2)


def get_info(np, vs):
    #cargos
    cargos = vs.find('tr', {'id': f'ctl00_containerCorpo_grdData_DXDataRow{np}'})

    format_text(cargos)

    #pessoas
    z = vs.find('td', {'id': f'ctl00_containerCorpo_grdData_tcdxdt{np}'})
    z = z.get_text().split("\n")

    global valores_seis_digitos 
    valores_seis_digitos = [valor for valor in z if valor[:6].isdigit()]
    sleep(2)

def click_boton(n):
    botoes = navegador.find_elements_by_css_selector('td > button > span')
    botoes[n].click()

#tabela = pd.DataFrame(columns=['Matrícula', 'Nome', 'CPF', 'Contrato', 'Data', 'Status', 'Cargo e Nivel', 'Salario'])

for i in range(201,239):
    click_boton(i - 200)
    sleep(5)

    x = BeautifulSoup(navegador.page_source, 'html.parser')

    sleep(2)

    MAX_CLICKS = 10
    click_count = 0

    while True:
        try:
            elementos = navegador.find_elements_by_css_selector('a[data-args="PBN"]')
            # Exclui o último elemento da lista
            elementos = elementos[:-1]
            
            elementos[0].click()

            x = BeautifulSoup(navegador.page_source, 'html.parser')

            sleep(2)

            get_info(i, x)

            for item in valores_seis_digitos:
                codigo = re.findall(r'^\d{6}', item)[0]
                nome = re.findall(r'\d{6}([A-Z\s]+)\*{3}', item)[0].strip()
                cpf = re.search(r'\*\*\*\.(\d{3})\.\d{3}\-\*\*', item)[0]
                contrato = list(re.search(r'\*\*\*\.(\d{3})\.\d{3}\-\*\*(.+?)(\d{2}/\d{2}/\d{4})', item).span())
                contrato[0] += 14
                contrato[1] -= 10
                data = re.findall(r'(\d{2}/\d{2}/\d{4})', item)[0]
                if '\xa0' in item[0]:
                    status = re.findall(r'(\xa0\w+)', item)[0].strip()

                padrao = r'(\w+)\s+(\d+-\d+-\d+)\s+(R\$\s[\d.,]+)'
                

                tabela = tabela.append({
                    'Matrícula': codigo,
                    'Nome': nome,
                    'CPF': cpf,
                    'Contrato': item[contrato[0]:contrato[1]],
                    'Data': data,
                    'Status': status
                }, ignore_index=True)

            tabela2 = tabela.tail(len(valores_seis_digitos))

            tabela2.loc[:, 'Salario'] = texto_formatado[1]
            tabela2.loc[:, 'Cargo e Nivel'] = texto_formatado[0]

            sleep(3)
                    
            click_count += 1
            if click_count >= MAX_CLICKS:
                break
        except (NoSuchElementException, IndexError):
            break

    sleep(10)

    get_info(i, x)

    sleep(3)

    for item in valores_seis_digitos:
        codigo = re.findall(r'^\d{6}', item)[0]
        nome = re.findall(r'\d{6}([A-Z\s]+)\*{3}', item)[0].strip()
        cpf = re.search(r'\*\*\*\.(\d{3})\.\d{3}\-\*\*', item)[0]
        contrato = list(re.search(r'\*\*\*\.(\d{3})\.\d{3}\-\*\*(.+?)(\d{2}/\d{2}/\d{4})', item).span())
        contrato[0] += 14
        contrato[1] -= 10
        data = re.findall(r'(\d{2}/\d{2}/\d{4})', item)[0]
        if '\xa0' in item[0]:
            status = re.findall(r'(\xa0\w+)', item)[0].strip()

        padrao = r'(\w+)\s+(\d+-\d+-\d+)\s+(R\$\s[\d.,]+)'
        

        tabela = tabela.append({
            'Matrícula': codigo,
            'Nome': nome,
            'CPF': cpf,
            'Contrato': item[contrato[0]:contrato[1]],
            'Data': data,
            'Status': status
        }, ignore_index=True)

    tabela2 = tabela.tail(len(valores_seis_digitos))

    tabela2.loc[:, 'Salario'] = texto_formatado[1]
    tabela2.loc[:, 'Cargo e Nivel'] = texto_formatado[0]

    sleep(3)

/tmp/ipykernel_9437/2464372537.py:116: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append({
/tmp/ipykernel_9437/2464372537.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabela2.loc[:, 'Salario'] = texto_formatado[1]
/tmp/ipykernel_9437/2464372537.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabela2.loc[:, 'Cargo e Nivel'] = texto_formatado[0]
/tmp/ipykernel_9437/2464372537.py:116: Futu

In [137]:
tabela.to_csv("/home/sdna/Documents/Paulo/tabela_portal_trans_2023")

In [102]:
tabela_real

,Matrícula,Nome,CPF,Contrato,Data,Status,Cargo e Nivel,Salario
0,768452,LARA ALMEIDA DE OLIVEIRA,***.431.696-**,Contrato,16/02/2022,Ativo,ADVOGADO 01-01-50,"R$ 1.500,00"
1,768699,ADSON SOUSA CASTRO,***.305.616-**,Contrato,01/12/2022,Ativo,AG COMUNITARIO PSF 01-01-468,"R$ 1.550,00"
2,768028,ALEX MARINHO SOARES DA SILVA,***.045.046-**,Contrato,09/08/2021,Ativo,AG COMUNITARIO PSF 01-01-468,"R$ 1.550,00"
3,768029,ALINE RAMOS DE OLIVEIRA,***.132.446-**,Contrato,18/08/2021,Ativo,AG COMUNITARIO PSF 01-01-468,"R$ 1.550,00"
4,768558,AMANDA DE AMORIM,***.970.726-**,Contrato,01/05/2022,Ativo,AG COMUNITARIO PSF 01-01-468,"R$ 1.550,00"
...,...,...,...,...,...,...,...,...
944,764072,VIRGINIA NUNES SOARES RODRIGUES,***.918.436-**,Efetivo,01/02/2012,Ativo,PROF ED BASICA II PII-II-B,"R$ 22,43"
945,763639,CLEO RAMOS DA SILVA,***.478.716-**,Efetivo,01/12/2010,Ativo,PROF ED BASICA II PII-II-D,"R$ 23,71"
946,764068,KARINE MONTEIRO PEREIRA,***.181.926-**,Efetivo,01/02/2012,Ativo,PROF ED BASICA II PII-II-D,"R$ 23,71"
947,763659,LUCILEIA ALVES SILVA,***.699.786-**,Efetivo,01/12/2010,Ativo,PROF ED BASICA II PII-II-D,"R$ 23,71"
